```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```

전체 구간에 대해 유전 알고리즘을 적용시킴니다.

In [1]:
# import 및 변수정의 구간
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

# 역 구간
Station_section = 11

# 하루에 운행하는 열차 수
One_day_train = 100

#교차율
Pc = 0.7

#변이율 역수 관계
Pm = 1000

#마지막 세대수 
largest_generation = 30

#보존할 상위값
save_value = 5


In [2]:
# 01010로 이루어진 리스트를 10진수로 디코딩하는 함수

def decoding(X, Y=0):
    """
    input   : 0100로 이루어진 리스트
    return : 10진수 정수
    """
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [3]:
#파일을 2차원 배열로 바꿔주기

def Readtxt(fname):
    """
    input  : 파일들
    return : 2차원 배열 
    """
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [4]:
# 바뀌야됨

def ScoreFunction(X):
    """
    X에 속도 리스트를 넣어서 평가
    """
    AirportNum = Readtxt('data/Number-of-train-people.txt')
    AirportTime = Readtxt('data/Time-of-airport-train.txt')
    AtherTime = Readtxt('data/Time-of-ather-train.txt')
    sp = decoding(X)
    return (777 - (6 * sp)) * 8

In [5]:
#초기값
speedlist = [[[[random.choice([0,1]) for i in range(listN)]for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]



In [6]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]),len(speedlist[0][0][0]))

30 100 11 8


In [7]:
#하루 열차의 총합 구하기
def Make_Score_per_all(speedlist):
    #열차마다 점수                             #30개의 경쟁 100개의 열차
    Score_per_train = [[0 for i in range(One_day_train)]for j in range(N)]

    # 30개의 경쟁
    for k in range(N):
        # 100대의 열차
        for i in range(One_day_train):
            # 11개의 역
            for j in range(Station_section):
                #각 열차에 대해      += 11개의 구간의 점수 합                   ([30경쟁][100대열차] +=  [30경쟁][100대열차][11개의 역])
                Score_per_train[k][i] += ScoreFunction(speedlist[k][i][j])


    # 하루마다의 점수 
    Score_per_all = [0 for i in range(N)]

    # 30개의 경쟁            
    for i in range(N):
        # 100대의 열차
        for j in range(One_day_train):
             Score_per_all[i] += Score_per_train[i][j]
                
    return Score_per_all

In [63]:
Score_per_all = Make_Score_per_all(speedlist)
Score_per_all

[6890016.0,
 6826848.0,
 6682800.0,
 6813072.0,
 6787728.0,
 6849072.0,
 6825408.0,
 6931632.0,
 7021968.0,
 6857040.0,
 6862848.0,
 7103472.0,
 6861936.0,
 6753888.0,
 6791184.0,
 6667008.0,
 6824592.0,
 6766320.0,
 6908448.0,
 7098576.0,
 6893904.0,
 6915264.0,
 6928800.0,
 6690384.0,
 6906192.0,
 6930576.0,
 6918672.0,
 6946272.0,
 6873024.0,
 6857808.0]

In [64]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]), len(speedlist[0][0][0]),"총개수 : " ,len(speedlist) * len(speedlist[0]) * len(speedlist[0][0]) * len(speedlist[0][0][0])
)

print("-"*100)

#하루마다의 점수 (이걸로 경쟁)
print(Score_per_all[1])
#열차마다의 점수
#print(Score_per_train[1][2])
#구간마다의 점수
print(ScoreFunction(speedlist[1][2][1]))


30 100 11 8 총개수 :  264000
----------------------------------------------------------------------------------------------------
6826848.0
6120.0


In [8]:
def Score_assessment_group(Score_per_all):
    # 적합도 비율 찾기
    speedlist_evaluation = [[Score_per_all[i], 0, i] for i in range(N)]

    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += Score_per_all[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][1] = speedlist_evaluation[i][0]/Sum_of_fitness
        else:
            speedlist_evaluation[i][1] = speedlist_evaluation[i-1][1]  + speedlist_evaluation[i][0]/Sum_of_fitness
    
    return speedlist_evaluation


In [66]:
#평가한 값 
Group_assessment_value = Score_assessment_group(Score_per_all)

#                                         정렬   점수값이 높은 순으로
sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
#상위 save_value 개값        
save_assessmend_value = sortand_Group_assessmend[0:save_value]
# Score_per_all에서 몇번 배열이 상위 5개인지만 
# 리스트 해쉬값만
save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]

print(save_assessmend_num)

print(Score_per_all[save_assessmend_num[0]])

Group_assessment_value

[11, 19, 8, 27, 7]
7103472.0


[[6890016.0, 0.03344915549865555, 0],
 [6826848.0, 0.06659164752156024, 1],
 [6682800.0, 0.09903482564573518, 2],
 [6813072.0, 0.13211043893190696, 3],
 [6787728.0, 0.1650630139846468, 4],
 [6849072.0, 0.1983133974887617, 5],
 [6825408.0, 0.23144889870294866, 6],
 [6931632.0, 0.26510008857354644, 7],
 [7021968.0, 0.29918983517770287, 8],
 [6857040.0, 0.3324789011567225, 9],
 [6862848.0, 0.3657961633975703, 10],
 [7103472.0, 0.40028158977514994, 11],
 [6861936.0, 0.43359442450380986, 12],
 [6753888.0, 0.46638271555168315, 13],
 [6791184.0, 0.49935206854534553, 14],
 [6667008.0, 0.5317185808005828, 15],
 [6824592.0, 0.5648501205564963, 16],
 [6766320.0, 0.5976987655862993, 17],
 [6908448.0, 0.6312374034365418, 18],
 [7098576.0, 0.6656990610644812, 19],
 [6893904.0, 0.6991670917466746, 20],
 [6915264.0, 0.7327388194248475, 21],
 [6928800.0, 0.7663762607049671, 22],
 [6690384.0, 0.7988562570883887, 23],
 [6906192.0, 0.8323839426716401, 24],
 [6930576.0, 0.8660300059491782, 25],
 [6918672.0

In [9]:
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][1]:
                break
        if Group_assessment_value[i-1][1] < roulette < Group_assessment_value[i][1]:
            break
    return i

In [68]:
Selected_list_in_roulette = []

#룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
for i in range(N-5):
    Selected_list_in_roulette.append(roulettes(Group_assessment_value))

Selected_list_in_roulette += save_assessmend_num


Selected_list_in_roulette

[26,
 4,
 29,
 22,
 13,
 8,
 3,
 1,
 14,
 3,
 13,
 5,
 22,
 20,
 15,
 21,
 6,
 18,
 19,
 23,
 26,
 26,
 14,
 0,
 26,
 11,
 19,
 8,
 27,
 7]

In [69]:
len(Selected_list_in_roulette)

30

In [10]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = [[[0 for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]
    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        for j in range(One_day_train):
            for k in range(Station_section):
                #교차
                selection = random.random()
                ###############################################################
                if selection < Pc:
                    rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
                    # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k][0:rand]         + speedlist[Selected_list_in_roulette[i*2+1]][j][k][rand:]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k][0:rand] + speedlist[Selected_list_in_roulette[i*2]][j][k][rand:]
                else:
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k]
                #변의
                for z in range(2):
                    # Pm * 100 분의 1의 확률 
                    if randint(0, 100 * Pm) == 0:
                        #selection 부분을 변의 1 -> 0  0 -> 1
                        if W[i*2+z][j][k][rand] == 0:
                            W[i*2+z][j][k][rand] = 1
                        else:
                            W[i*2+z][j][k][rand] = 0
    return W

In [71]:
test_new_lest = Change_parent(speedlist, Selected_list_in_roulette)

In [72]:
print(len(test_new_lest), len(test_new_lest[0]), len(test_new_lest[0][0]),len(test_new_lest[0][0][0]))

30 100 11 8


In [73]:
test_Score_per_all = Make_Score_per_all(test_new_lest)
test_Score_per_all

[6982512.0,
 6774096.0,
 6850704.0,
 6955296.0,
 6783840.0,
 7007664.0,
 6838224.0,
 6784512.0,
 6819696.0,
 6867408.0,
 6783600.0,
 6900672.0,
 6916560.0,
 6869952.0,
 6664176.0,
 6861360.0,
 6866016.0,
 6977568.0,
 7089120.0,
 6697344.0,
 6918672.0,
 6918672.0,
 6768432.0,
 6915264.0,
 6952416.0,
 7106016.0,
 7094160.0,
 7005360.0,
 6968160.0,
 6900720.0]

In [13]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        #전체 값으로 해서 
        Score_per_all = Make_Score_per_all(speedlist)

        #평가한 값 
        Group_assessment_value = Score_assessment_group(Score_per_all)

        #                                         정렬   점수값이 높은 순으로
        sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
        #상위 save_value 개값        
        save_assessmend_value = sortand_Group_assessmend[0:save_value]
        # Score_per_all에서 몇번 배열이 상위 5개인지만 
        # 리스트 해쉬값만
        save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]


        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
        for i in range(N-5):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))

        Selected_list_in_roulette += save_assessmend_num
        
        #룰렛 랜덤하게  근데이거하면 오류남 왜지?
        #Selected_list_in_roulette = random.shuffle(Selected_list_in_roulette)
        # ------------------------------------------------------------------------------------------------------------------------------------------------------------
        
        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        
        New_Score_per_all = Make_Score_per_all(New_list)
        
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_Score_per_all):
            print(i, nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

In [14]:
test_list = []

test_list = Evolution(speedlist, 0)

---------------
0 번째를 합니다.
---------------
0 6682320.0
1 6892560.0
2 7010208.0
3 6932928.0
4 6946080.0
5 6764400.0
6 6757200.0
7 6625968.0
8 6871920.0
9 6730944.0
10 6792672.0
11 6921840.0
12 6808464.0
13 6979296.0
14 6757920.0
15 6783504.0
16 6919680.0
17 6709248.0
18 6739920.0
19 6852528.0
20 6796320.0
21 6752688.0
22 6903120.0
23 6955488.0
24 6995616.0
25 7012080.0
26 7033344.0
27 6976416.0
28 6975696.0
29 6933168.0
---------------
1 번째를 합니다.
---------------
0 6685488.0
1 6963984.0
2 6907344.0
3 6828144.0
4 6764112.0
5 7107120.0
6 6894960.0
7 6803520.0
8 6716256.0
9 6919872.0
10 6800208.0
11 6732816.0
12 6987024.0
13 6718656.0
14 6717168.0
15 7069296.0
16 7052688.0
17 6967872.0
18 6731952.0
19 6751728.0
20 6889056.0
21 7011072.0
22 6956832.0
23 6783984.0
24 6792144.0
25 7018320.0
26 6960672.0
27 7004016.0
28 6995952.0
29 6954768.0
---------------
2 번째를 합니다.
---------------
0 6960768.0
1 6771168.0
2 6892224.0
3 6751920.0
4 6930864.0
5 6947520.0
6 6770304.0
7 6879264.0
8 6752064.0
9 6

---------------
20 번째를 합니다.
---------------
0 7137504.0
1 7144752.0
2 7150752.0
3 7165056.0
4 7202496.0
5 7139328.0
6 7134864.0
7 7134864.0
8 7130496.0
9 7142592.0
10 7146192.0
11 7180992.0
12 7177008.0
13 7156272.0
14 7186896.0
15 7182192.0
16 7204416.0
17 7142064.0
18 7121952.0
19 7173840.0
20 7165104.0
21 7172208.0
22 7185744.0
23 7186176.0
24 7195680.0
25 7219728.0
26 7210368.0
27 7208208.0
28 7199904.0
29 7205232.0
---------------
21 번째를 합니다.
---------------
0 7183440.0
1 7155456.0
2 7219776.0
3 7205184.0
4 7183008.0
5 7167792.0
6 7164576.0
7 7145136.0
8 7152816.0
9 7154256.0
10 7196208.0
11 7155744.0
12 7135152.0
13 7206528.0
14 7096464.0
15 7178784.0
16 7206864.0
17 7131840.0
18 7152720.0
19 7149840.0
20 7171056.0
21 7191696.0
22 7210128.0
23 7181808.0
24 7143408.0
25 7211184.0
26 7210224.0
27 7208352.0
28 7205472.0
29 7204176.0
---------------
22 번째를 합니다.
---------------
0 7163760.0
1 7185168.0
2 7139520.0
3 7166688.0
4 7208256.0
5 7149936.0
6 7154208.0
7 7212720.0
8 7216176.0


In [15]:
test_list

[[[[1, 0, 1, 0, 1, 0, 0, 0],
   [0, 1, 1, 0, 0, 0, 0, 0],
   [1, 1, 1, 0, 0, 1, 0, 0],
   [0, 1, 1, 0, 0, 0, 1, 0],
   [1, 1, 0, 1, 0, 0, 0, 1],
   [0, 0, 1, 0, 1, 0, 0, 0],
   [1, 0, 0, 0, 1, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 1],
   [1, 0, 0, 0, 1, 1, 0, 1],
   [1, 0, 0, 1, 1, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 0, 1]],
  [[1, 1, 1, 1, 1, 1, 0, 1],
   [0, 1, 0, 0, 0, 1, 0, 0],
   [1, 1, 1, 0, 1, 1, 0, 0],
   [1, 0, 1, 1, 1, 0, 1, 1],
   [1, 0, 0, 1, 1, 0, 0, 0],
   [1, 0, 0, 0, 1, 0, 0, 0],
   [1, 0, 0, 0, 0, 1, 0, 1],
   [0, 1, 1, 1, 1, 0, 0, 0],
   [0, 1, 1, 0, 0, 1, 1, 0],
   [0, 0, 1, 0, 1, 0, 0, 1],
   [0, 1, 1, 0, 1, 0, 1, 0]],
  [[0, 0, 1, 1, 0, 1, 1, 0],
   [1, 1, 0, 0, 1, 0, 1, 1],
   [1, 1, 0, 0, 0, 0, 0, 1],
   [0, 0, 1, 1, 0, 1, 1, 1],
   [0, 1, 0, 1, 1, 1, 0, 1],
   [0, 0, 1, 0, 1, 1, 1, 0],
   [0, 0, 0, 1, 0, 1, 0, 0],
   [0, 1, 1, 0, 1, 0, 1, 1],
   [0, 1, 1, 1, 0, 0, 1, 0],
   [0, 1, 1, 0, 1, 1, 0, 0],
   [0, 1, 1, 1, 0, 0, 1, 1]],
  [[1, 0, 0, 1, 0, 1, 0, 0],
   [1, 0, 1

In [51]:
if speedlist[1][1][1][1] == 0:
    print("!!")
else:
    print("??")

??


In [37]:
Www = [[[0 for i in range(2)]for j in range(2)]for k in range(2)]
    # 교차함수 및 변이함수 (자식만들기)
    
print(Www)
for i in range(2):
    for j in range(2):
        for k in range(2):
            Www[i][j][k] = [43,43,0,2,2]
            
Www

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


[[[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]],
 [[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]]]